In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [2]:
df = pd.read_csv('iris.csv')
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
cat_cols = pd.DataFrame(ohe.fit_transform(df[['species']]).toarray(),columns = df['species'].unique())
df = pd.concat([df, cat_cols], axis=1)
df.drop('species',axis=1, inplace=True)

In [6]:
json_file = pd.read_json('algoparams_from_ui.json')
json_file

,a,design_state_data,session_description,session_name
algorithms,b,{'DecisionTreeClassifier': {'is_selected': Fal...,test,test
feature_generation,b,{'explicit_pairwise_interactions': ['sepal_wid...,test,test
feature_handling,b,{'petal_length': {'feature_details': {'impute_...,test,test
feature_reduction,b,"{'depth_of_trees': '6', 'feature_reduction_met...",test,test
hyperparameters,b,{'cross_validation_stratergy': 'Time-based K-f...,test,test
metrics,b,"{'compute_lift_at': 0, 'cost_matrix_gain_for_f...",test,test
probability_calibration,b,{'probability_calibration_method': 'Sigmoid - ...,test,test
session_info,b,"{'dataset': 'iris_modified.csv', 'experiment_i...",test,test
target,b,"{'partitioning': True, 'prediction_type': 'Reg...",test,test
train,b,"{'k_fold': False, 'policy': 'Split the dataset...",test,test


In [ ]:
task 1 Reading target file

In [7]:
target = json_file.loc['target','design_state_data']['target']
type_of_reg = json_file.loc['target','design_state_data']['type']

In [8]:
feature_handling_dic= json_file.loc['feature_handling','design_state_data']
feature_handling_dic

{'petal_length': {'feature_details': {'impute_value': 0,
   'impute_with': 'Average of values',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling'},
  'feature_name': 'petal_length',
  'feature_variable_type': 'numerical',
  'is_selected': True},
 'petal_width': {'feature_details': {'impute_value': -2,
   'impute_with': 'custom',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling'},
  'feature_name': 'petal_width',
  'feature_variable_type': 'numerical',
  'is_selected': True},
 'sepal_length': {'feature_details': {'impute_value': 0,
   'impute_with': 'Average of values',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling'},
  'feature_name': 'sepal_length',
  'feature_variable_

In [9]:
def feature_handling(feature_handling, column_names,df):
    for col in column_names:
        try:
            if feature_handling[col]['feature_details']['impute_with'] == 'custom':
                df[col] = df[col].fillna(feature_handling[col]['feature_details']['impute_value'])
            elif feature_handling[col]['feature_details']['impute_with'] == 'Average of values': 
                df[col] = df[col].fillna(df[col].mean())
        except KeyError:
            print(col,"feature does not have 'impute_with' provided")
    return df

In [10]:
feature_handling(feature_handling_dic, df.columns, df)

Iris-setosa feature does not have 'impute_with' provided
Iris-versicolor feature does not have 'impute_with' provided
Iris-virginica feature does not have 'impute_with' provided


,sepal_length,sepal_width,petal_length,petal_width,Iris-setosa,Iris-versicolor,Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


CREATEING MODEL OBJECTIVE

In [11]:
type_of_reg

'regression'

In [12]:
json_file.loc['algorithms']['design_state_data']

{'DecisionTreeClassifier': {'is_selected': False,
  'max_depth': 7,
  'min_depth': 4,
  'min_samples_per_leaf': [12, 6],
  'model_name': 'Decision Tree',
  'use_best': True,
  'use_entropy': True,
  'use_gini': False,
  'use_random': True},
 'DecisionTreeRegressor': {'is_selected': False,
  'max_depth': 7,
  'min_depth': 4,
  'min_samples_per_leaf': [12, 6],
  'model_name': 'Decision Tree',
  'use_best': True,
  'use_entropy': True,
  'use_gini': False,
  'use_random': True},
 'ElasticNetRegression': {'is_selected': False,
  'max_elasticnet': 0.8,
  'max_iter': 50,
  'max_regparam': 0.8,
  'min_elasticnet': 0.5,
  'min_iter': 30,
  'min_regparam': 0.5,
  'model_name': 'Lasso Regression',
  'regularization_term': 'Specify values to test'},
 'GBTClassifier': {'feature_sampling_statergy': 'Fixed number',
  'fixed_number': 22,
  'is_selected': False,
  'learningRate': [],
  'max_depth': 7,
  'max_iter': 40,
  'max_stepsize': 0.5,
  'max_subsample': 2,
  'min_depth': 5,
  'min_iter': 20,
  

In [ ]:
LI

In [13]:
from sklearn.linear_model import LinearRegression
LinearRegression: {'model_name': 'LinearRegression',
  'is_selected': False,
  'parallelism': 2,
  'min_iter': 30,
  'max_iter': 50,
  'min_regparam': 0.5,
  'max_regparam': 0.8,
  'min_elasticnet': 0.5,
  'max_elasticnet': 0.8}
lin_reg = LinearRegression()
lin_reg.fit(df.drop(target, axis=1), df[target])
lin_reg.predict(df.drop(target, axis=1))

array([ 0.24014568,  0.1342577 ,  0.1790507 ,  0.21174828,  0.27458958,
        0.38387418,  0.26255738,  0.24874845,  0.1566694 ,  0.18332048,
        0.28574862,  0.29179513,  0.14373364,  0.11882887,  0.25046437,
        0.4320966 ,  0.28749494,  0.24014568,  0.33047841,  0.33914439,
        0.25903434,  0.31417643,  0.21611408,  0.26249417,  0.36407956,
        0.17297139,  0.27284326,  0.25476455,  0.20570178,  0.25133513,
        0.21689122,  0.21084472,  0.40457237,  0.37701772,  0.18332048,
        0.12652809,  0.17814713,  0.18332048,  0.15754256,  0.23927252,
        0.2255268 , -0.02670915,  0.2074785 ,  0.29781123,  0.43552363,
        0.14373364,  0.3632392 ,  0.21262144,  0.29522456,  0.19968567,
        1.43855549,  1.44722148,  1.47125308,  1.18731911,  1.36196848,
        1.41368114,  1.529855  ,  1.10047901,  1.37746051,  1.29152398,
        1.03932083,  1.37238078,  1.11497148,  1.44893499,  1.23127175,
        1.3697309 ,  1.47309301,  1.282858  ,  1.21649366,  1.20